In [2]:
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [3]:
raw_consumer_fraud_df = pd.read_csv("../data/tables/consumer_fraud_probability.csv")

In [4]:
raw_consumer_fraud_df

,user_id,order_datetime,fraud_probability
0,6228,2021-12-19,97.629808
1,21419,2021-12-10,99.247380
2,5606,2021-10-17,84.058250
3,3101,2021-04-17,91.421921
4,22239,2021-10-19,94.703425
...,...,...,...
34859,18466,2022-02-22,8.679873
34860,17552,2021-12-26,8.349463
34861,17875,2021-06-27,8.288847
34862,10401,2021-09-18,8.842216


In [5]:
raw_merchant_fraud_df = pd.read_csv("../data/tables/merchant_fraud_probability.csv").sort_values('fraud_probability')
raw_merchant_fraud_df.sort_values('fraud_probability')

,merchant_abn,order_datetime,fraud_probability
21,99989036621,2021-10-17,18.210891
37,67264251405,2021-10-19,18.210891
44,83199298021,2021-12-14,22.799803
34,83199298021,2022-01-04,23.920306
31,83199298021,2021-12-30,23.998597
...,...,...,...
6,23686790459,2021-12-10,79.454344
57,14530561097,2021-09-15,80.800545
65,97884414539,2021-10-19,89.799200
41,35575706403,2022-02-20,91.096068


In [6]:
consumer_id = pd.read_csv('../data/curated/consumer.csv')
consumer_id

,name,address,state,postcode,gender,consumer_id,Median_tot_prsnl_inc_weekly
0,Mary Smith,3764 Amber Oval,NSW,2782,Female,179208,740.0
1,Nicholas Peck,32589 Foster Park Apt. 749,NSW,2782,Male,729157,740.0
2,Paul Hoffman,7455 Tara Village,NSW,2782,Male,883492,740.0
3,Andrea Johnson,9204 West Isle,NSW,2782,Female,275781,740.0
4,Steven Martin,8559 Laura Way Apt. 740,NSW,2782,Male,890359,740.0
...,...,...,...,...,...,...,...
416813,Mr. Andrew Andersen,246 Morris Mills,QLD,4343,Male,732378,656.0
416814,Emily Smith,4092 Cummings Forge,QLD,4343,Female,1053557,656.0
416815,Stacey Johnson,591 Patrick Wall Apt. 544,QLD,4343,Female,1069993,656.0
416816,Sherry Wells,9864 Shawn Overpass Apt. 755,QLD,4343,Female,1056856,656.0


In [7]:
raw_consumer_df = pd.read_parquet('../data/tables/consumer_user_details.parquet')

In [8]:
consumer_df = raw_consumer_df.merge(consumer_id, on='consumer_id', how='inner')

In [9]:
consumer_fraud_df = raw_consumer_fraud_df.merge(consumer_df, on = 'user_id', how='inner')[['user_id', 'order_datetime', 'gender', 'Median_tot_prsnl_inc_weekly', 'fraud_probability']]
consumer_fraud_df

,user_id,order_datetime,gender,Median_tot_prsnl_inc_weekly,fraud_probability
0,6228,2021-12-19,Male,1259.0,97.629808
1,6228,2021-12-19,Male,1259.0,97.629808
2,6228,2022-01-31,Male,1259.0,18.045230
3,5606,2021-10-17,Undisclosed,920.0,84.058250
4,5606,2021-10-17,Undisclosed,920.0,84.058250
...,...,...,...,...,...
29078,20347,2022-01-03,Male,824.0,8.310974
29079,33,2022-02-20,Female,735.0,8.318253
29080,17875,2021-06-27,Male,880.0,8.288847
29081,10401,2021-09-18,Male,629.0,8.842216


In [10]:
merchant_detail = pd.read_parquet('../data/curated/merchant_detail.parquet')

FileNotFoundError: [Errno 2] No such file or directory: '../data/curated/merchant_detail.parquet'

In [ ]:
merchant_fraud_df = raw_merchant_fraud_df(merchant_detail, on = 'abn', how = 'inner')